In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

import torch
from bertopic import BERTopic
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sentence_transformers import SentenceTransformer

from datetime import datetime
from tqdm.auto import tqdm
import html

sns.set(palette='summer')

## Load data

In [160]:
data = pd.read_json('../data/geo-reviews-dataset-2023.jsonl', lines=True)

In [161]:
data.head(2)

,address,name_ru,rating,rubrics,text,#_words,#_chars
0,"Республика Мордовия, Ковылкино, улица Ленина, 2А",СберБанк,5,Банк,"Всё отлично, вежливый персонал, талонная систе...",34,235
1,"Санкт-Петербург, улица Коллонтай, 12, корп. 1",Пятёрочка,5,Супермаркет,"Чистота, без просроченных продуктов, персонал ...",8,80


In [162]:
len(data)

19137

In [163]:
data["name_ru"].value_counts()

Пятёрочка           6030
Магнит              2611
Красное&Белое       1732
Wildberries         1698
Ozon                1494
Вкусно — и точка    1181
Перекрёсток         1156
Fix Price           1112
Пляж                1091
СберБанк            1032
Name: name_ru, dtype: int64

In [164]:
data = data[data["name_ru"] == "Пятёрочка"]

## Simple preprocessing

In [165]:
data = data[data['#_words'] > 4]
data['text'] = data['text'].apply(
    lambda x: " ".join(x.lower().replace('ё', 'е').strip().split())
)

data['text'] = data['text'].apply(html.unescape)
len(data)

5947

In [166]:
data = data[data["rating"].apply(lambda x: x not in [5])]

In [167]:
len(data)

1692

In [173]:
data["#_words"].mean()

38.336288416075654

## Sentiment analysis

https://huggingface.co/Tatyana/rubert-base-cased-sentiment-new

In [12]:
model = AutoModelForSequenceClassification.from_pretrained('Tatyana/rubert-base-cased-sentiment-new')
tokenizer = AutoTokenizer.from_pretrained('Tatyana/rubert-base-cased-sentiment-new')

In [13]:
scores = []
batch_size = 4
device = 'cpu'

model = model.to(device)
model.eval()

with torch.no_grad():
    for i in tqdm(range(0, len(data), batch_size)):
        batch = tokenizer(
            data['text'].iloc[i:i+batch_size].tolist(), 
            truncation=True, padding='longest', return_tensors='pt', max_length=512)
        batch = {k: v.to(device) for k, v in batch.items()}
        
        preds = model(**batch).logits.argmax(dim=1)
        scores.append(preds)

  0%|          | 0/212 [00:00<?, ?it/s]

In [14]:
preds_mapping = {
    0: "NEUTRAL",
    1: "POSITIVE",
    2: "NEGATIVE",
}

In [15]:
data['sentiment'] = torch.cat(scores).tolist()
data['sentiment'] = data['sentiment'].apply(lambda x: preds_mapping[x])

In [16]:
data['sentiment'].value_counts()

NEGATIVE    483
POSITIVE    213
NEUTRAL     151
Name: sentiment, dtype: int64

## Clusterization

In [60]:
def get_clusters(sample_data: pd.DataFrame) -> pd.DataFrame:
    bert = SentenceTransformer('distiluse-base-multilingual-cased-v1')
    bert.eval()
    topic_model = BERTopic(embedding_model=bert, verbose=True, min_topic_size=10, calculate_probabilities=True)
    
    embeddings = bert.encode(sample_data['text'].tolist(), show_progress_bar=True)
    topics, probs = topic_model.fit_transform(sample_data['text'].tolist(), embeddings=embeddings)
    
    sample_data['topics'] = topics
    
    topic_info = topic_model.get_topic_info()[['Topic', 'Name']]
    topic_info.columns = ['topics', 'keywords']
    topic_info['keywords'] = topic_info['keywords'].apply(lambda x: ", ".join(x.split('_')[1:]))
    
    sample_data = sample_data.merge(topic_info, how='left', on='topics')
    
    return sample_data

In [61]:
data = data.drop_duplicates(subset=["text"])

In [62]:
clusters = get_clusters(data)

Batches:   0%|          | 0/27 [00:00<?, ?it/s]

2023-12-19 23:24:25,736 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-19 23:24:28,512 - BERTopic - Dimensionality - Completed ✓
2023-12-19 23:24:28,513 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-19 23:24:28,567 - BERTopic - Cluster - Completed ✓
2023-12-19 23:24:28,571 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-19 23:24:28,665 - BERTopic - Representation - Completed ✓


In [63]:
clusters["topics"].value_counts()

-1     317
 0      92
 1      70
 2      56
 3      47
 4      44
 5      37
 6      32
 7      31
 8      22
 9      21
 10     19
 11     18
 12     15
 13     13
 14     13
Name: topics, dtype: int64

### Analysis with ChatGPT

Получение осмысленных коротких описаний для отдельных кластеров

In [79]:
import openai

In [133]:
openai.api_key = "..."

In [141]:
def get_topic_summary(df, topic_id, total_samples=7, model="gpt-3.5-turbo") -> (str, str):
    keywords = df[df['topics'] == topic_id]["keywords"].iloc[0]
    prompt = "Какие выводы можно сделать по этиму кластеру отзывов пользователей?\n\n"
    prompt += f"Ключевые слова для кластера: {keywords}\n\n"

    for i, x in enumerate(df[df['topics'] == topic_id]['text'].sample(total_samples, random_state=42)):
        prompt += f"{i+1}. {x.strip()}.\n"

    prompt += "В ответ напиши один ключевой вывод."
    #print(prompt)
    
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    
    answer = response.choices[0].message.content
    
    ask_help = """А теперь предложи возможнjе решение для этой проблемы с подробным планом. 
    Что бы ты сделал на месте руководителя крупной ритейл сети Пятерочка? 
    Придумай одно конкретное решение и распиши подробные шаги для его реализации. \
    При этом опиши definition of done для каждого шага и проекта в целом, потенциальных участников, их мотивацию, а также на какие бизнесовые метрики нужно смотреть и как они могут улучшиться от этого решения."""
    
    solution = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": answer},
            {"role": "user", "content": ask_help}
        ]
    )
    
    solution = solution.choices[0].message.content
    
    return answer, solution

In [143]:
report = []
sentiment_values = ["POSITIVE", "NEGATIVE", "NEUTRAL"]


for topic_id in tqdm(sorted(clusters["topics"].unique())):
    #topic_representation = get_topic_summary(clusters, topic_id)
    
    sub_df = clusters[clusters["topics"] == topic_id]
    
    current_line = [topic_id, len(sub_df)]
    current_line.extend(
        [(sub_df["sentiment"] == sent).sum() / len(sub_df) for sent in sentiment_values]
    )
    
    report.append(current_line)

  0%|          | 0/16 [00:00<?, ?it/s]

In [144]:
report = pd.DataFrame(report)

In [145]:
report.columns = ["topic_id", "total"] + sentiment_values
report = report[report["NEGATIVE"] > 0.7]

In [146]:
representations = []
solutions = []

for topic_id in tqdm(report["topic_id"]):
    topic_representation, topic_solution = get_topic_summary(clusters, topic_id, model="gpt-4-turbo")
    
    representations.append(topic_representation)
    solutions.append(topic_solution)

  0%|          | 0/5 [00:00<?, ?it/s]

In [147]:
report["representation"] = representations
report["solution"] = solutions

In [148]:
report["representation"] = report["representation"].apply(lambda x: x.lstrip("Тема: ").strip())

In [149]:
for col in sentiment_values:
    report[col] = report[col].apply(lambda x: round(x, 2))

In [195]:
for i, (topic, solution, score) in enumerate(zip(report["representation"], report["solution"], report["NEGATIVE"])):
    print(i, score, topic)
    print("SOLUTION:", solution)
    print('-' * 10)

0 0.71 Один из ключевых выводов по данному кластеру отзывов пользователей состоит в том, что магазин имеет проблемы с ценниками, в результате чего клиентам приходится проверять чеки и сталкиваться с несовпадением цен на товары.
SOLUTION: Решение: Автоматизация мониторинга ценников в магазинах Пятерочка

Шаги для реализации:

1. Анализ и планирование проекта:
   - Определить цели и ожидаемые результаты проекта: снижение ошибок при выставлении ценников, улучшение опыта покупателей, увеличение доверия к магазину.
   - Определить бизнесовые метрики, на которые будем смотреть: процент ошибок в ценниках, уровень удовлетворенности клиентов, продажи товаров.
   - Сформировать команду проекта, включающую представителей отделов маркетинга, операций и IT.
   - Разработать план проекта с определением ролей и ответственностей, сроками и бюджетом.

2. Подготовка инфраструктуры:
   - Провести анализ и выбор подходящей системы для автоматизации мониторинга ценников.
   - Произвести интеграцию выбранно

In [170]:
report

,topic_id,total,POSITIVE,NEGATIVE,NEUTRAL,representation,solution
3,2,56,0.07,0.71,0.21,Один из ключевых выводов по данному кластеру о...,Решение: Автоматизация мониторинга ценников в ...
8,7,31,0.13,0.81,0.06,Один из ключевых выводов по этому кластеру отз...,Одно возможное решение для руководителя крупно...
9,8,22,0.00,1.00,0.00,Ключевой вывод: В данном кластере отзывов поль...,Решение: Внедрение программы обучения и тренин...
11,10,19,0.21,0.74,0.05,Один из ключевых выводов по этому кластеру отз...,Конкретное решение: Внедрение обновленной сист...
12,11,18,0.17,0.72,0.11,Один из ключевых выводов по этому кластеру отз...,Одно из возможных решений для улучшения ситуац...


In [171]:
report.to_csv("../data/final_report.csv", index=False)

# NPV Calculation

In [176]:
financial_effect = 70000
regular_costs = 3000
initial_costs = 200000
discount_rate = 0.16

cash_flow = financial_effect - regular_costs

In [194]:
npv = - initial_costs

for t in range(1, 31):
    npv += cash_flow / (1 + discount_rate)**t
    
    if t in [6, 12, 18, 24]:
        print(f"{t} months: {npv:.1f} rub")

6 months: 46877.3 rub
12 months: 148206.2 rub
18 months: 189795.8 rub
24 months: 206866.0 rub
